---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df= []
    file = open("university_towns.txt", "r")
    lines = file.read().split('\n')
    for line in lines:
        if line.endswith('[edit]'):
            state = line
        elif line != '':
            df.append([state,line])       
    df = pd.DataFrame(df,columns=["State", "RegionName"])
    df['State'] = df['State'].apply((lambda x : (x.split('['))[0]))
    df['RegionName'] = df['RegionName'].apply((lambda x : (x.split(' ('))[0]))
    return df

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('gdplev.xls',skiprows =219,usecols = [4,6],names = ['quarter', 'GDP'])
    for i in range (1,len(GDP['GDP'])-1):
        if(GDP['GDP'][i] < GDP['GDP'][i-1] and GDP['GDP'][i+1] < GDP['GDP'][i]):
            startqyear =  GDP['quarter'][i]
            break
    return startqyear

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('gdplev.xls',skiprows =219,usecols = [4,6],names = ['quarter', 'GDP'])
    for i in range (1,len(GDP['GDP'])-1):
        if(GDP['GDP'][i] < GDP['GDP'][i-1] and GDP['GDP'][i+1] < GDP['GDP'][i]):
            start = i+1
            break
    for i in range (start,len(GDP['GDP'])-1):
        if(GDP['GDP'][i] > GDP['GDP'][i-1] and GDP['GDP'][i+1] > GDP['GDP'][i]):
            endqyear = GDP['quarter'][i+1]
            break
    
    return endqyear
#check if the year at rise or the one after it

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('gdplev.xls',skiprows =219,usecols = [4,6],names = ['quarter', 'GDP'])
    for i in range (1,len(GDP['GDP'])-1):
        if(GDP['GDP'][i] < GDP['GDP'][i-1] and GDP['GDP'][i+1] < GDP['GDP'][i]):
            start = i
            break
    for i in range (start,len(GDP['GDP'])-1):
        if(GDP['GDP'][i] > GDP['GDP'][i-1] and GDP['GDP'][i+1] > GDP['GDP'][i]):
            end = i+1
            break
    return GDP.iloc[GDP.iloc[start:end]['GDP'].argmin()]['quarter']

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    homes = pd.read_csv('City_Zhvi_AllHomes.csv')
    columnstodrop = list(homes.columns)[3:51]
    columnstodrop.append('RegionID')
    homes.drop(columnstodrop,axis = 1,inplace = True)
    homes['State'] = homes['State'].replace(states)
    homes.set_index(['State','RegionName'],inplace = True)
    hcols = homes.columns
    for i in range(len(hcols)):
        if(hcols[i][-2:] == '01'):
            homes[(hcols[i][0:4]+'q1')] = np.mean(homes[hcols[i:i+3]],axis = 1)
        if(hcols[i][-2:] == '04'):
            homes[(hcols[i][0:4]+'q2')] = np.mean(homes[hcols[i:i+3]],axis = 1)
        if(hcols[i][-2:] == '07'):
            homes[(hcols[i][0:4]+'q3')] = np.mean(homes[hcols[i:i+3]],axis = 1)
        if(hcols[i][-2:] == '10'):
            homes[(hcols[i][0:4]+'q4')] = np.mean(homes[hcols[i:i+3]],axis = 1)
        coltodrop = homes.columns[0:200]
        df = homes.drop(coltodrop, axis = 1)
    return df

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    uni_towns = get_list_of_university_towns()
    houses_prices = convert_housing_data_to_quarters()
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    recession_before_start_index = (list(houses_prices.columns).index(recession_start))-1
    recession_before_start = houses_prices.columns[recession_before_start_index]
    houses_prices['price_ratio'] = houses_prices[recession_before_start]/houses_prices[recession_bottom]
    houses_prices.sort_index(inplace = True)
    unidf = pd.merge(houses_prices, uni_towns, how='inner', left_index=True, right_on=['State','RegionName'])
    unidf.set_index(['State','RegionName'],inplace = True)
    unidf.dropna(inplace = True)
    nonunidf = pd.concat([houses_prices,unidf],axis=0)   
    nonunidf = nonunidf.drop_duplicates(keep=False)
    nonunidf.dropna(inplace = True)
    statistic , pvalue = ttest_ind(unidf['price_ratio'], nonunidf['price_ratio'])
    if(pvalue < 0.01):
        diffrent = True
        better = 'university town'
    else:
        diffrent = False
        better = 'non-university town'    
    return (diffrent,pvalue,better)